In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [15]:
# Load files
Train = pd.read_csv('Train (1).csv')
Test = pd.read_csv('Test.csv')
SampleSubmission = pd.read_csv('SampleSubmission.csv')

C:\Users\hp\AppData\Local\Temp\ipykernel_10560\280896822.py:2: DtypeWarning: Columns (140,172,254,257,275,306,343,358,373,374,461,635,641,642,643) have mixed types. Specify dtype option on import or set low_memory=False.
  Train = pd.read_csv('Train (1).csv')
C:\Users\hp\AppData\Local\Temp\ipykernel_10560\280896822.py:3: DtypeWarning: Columns (172,252,254,257,275,343,358,359,461,641,642,643) have mixed types. Specify dtype option on import or set low_memory=False.
  Test = pd.read_csv('Test.csv')


In [16]:
Train.insert(2, 'paid_fees_proportion', Train['pri_fees_paid_proportion'].value_counts(normalize=True)[1])
Test.insert(2, 'paid_fees_proportion', Test['pri_fees_paid_proportion'].value_counts(normalize=True)[1])

Train.insert(3, 'avg_fees_by_age', Train.groupby('child_age')['pri_fees_amount'].transform('mean'))
Test.insert(3, 'avg_fees_by_age', Test.groupby('child_age')['pri_fees_amount'].transform('mean'))

Train.insert(4, 'height', Train['child_height'] < 200)
Test.insert(4, 'height', Test['child_height'] < 200)

Train.insert(5, 'pqa_teach', Train['pqa_score_teaching'] >= 8)
Test.insert(5, 'pqa_teach', Test['pqa_score_teaching'] >= 8)

Train.insert(6, 'pqa_environment_adequate', Train['pqa_score_environment'] >= 8)
Test.insert(6, 'pqa_environment_adequate', Test['pqa_score_environment'] >= 8)

Train.insert(7, 'child_age_5', Train['child_age'] < 60)
Test.insert(7, 'child_age_5', Test['child_age'] < 60)


In [17]:
""# Créer un nouveau attribut de ratio de livres accessibles
book_access_ratio = Train['obs_books'].apply(lambda x: 1 if x=='Yes' else 0) / 12000

# Créer un nouveau attribut de source d'eau potable
potable_water = Train['obs_water'].apply(lambda x: 1 if x=='Tap water in the building' else 0)

# Créer un nouveau attribut de ratio d'accès aux matériaux
material_access_ratio = Train['obs_accessible'].apply(lambda x: 1 if x=='Yes' else 0) / 12000

# Créer un nouveau attribut de ratio d'accès à l'eau potable
water_access_ratio = Train['obs_potable'].apply(lambda x: 1 if x=='Yes' else 0) / 12000


# Créer un nouveau attribut de ratio d'accès à l'énergie
energy_access_ratio = (Train['obs_heating_census'].apply(lambda x: 1 if x!='None' else 0) + \
                                Train['obs_cooking_census'].apply(lambda x: 1 if x!='None' else 0) + \
                                Train['obs_lighting_census'].apply(lambda x: 1 if x!='None' else 0)) / \
                               12000  

Train['teacher_duration'].fillna('', inplace=True)

# Define a function to create a new feature based on teacher_duration
def experience_bins(duration):
    if duration == '':
        return 'no experience'
    elif duration <= 2:
        return 'novice'
    elif duration <= 5:
        return 'intermediate'
    elif duration <= 10:
        return 'experienced'
    else:
        return 'expert'

"""
def social_rating_bins(rating):
    if np.isnan(rating):
        return 'unknown'
    elif rating <= 10:
        return 'low'
    elif rating <= 15:
        return 'medium'
    else:
        return 'high'


Train['obs_books_age'].fillna(0, inplace=True)

Train['obs_books'].fillna('Unknown', inplace=True)"""


    
teacher_experiencee = Train['teacher_duration'].apply(lambda x: experience_bins(x))

"""social_rating = Train['teacher_social_total'].apply(lambda x: social_rating_bins(x))

age_appropriate_books = Train['obs_books_age'].apply(lambda x: 1 if x == 'Yes' else 0)

books_available = Train['obs_books'].map({'Yes': 1, 'No': 0, 'Unknown': -1})"""







# Créer un nouveau attribut de ratio de livres accessibles
book_access_ratio = Test['obs_books'].apply(lambda x: 1 if x=='Yes' else 0) / 12000

# Créer un nouveau attribut de source d'eau potable
potable_water = Test['obs_water'].apply(lambda x: 1 if x=='Tap water in the building' else 0)

# Créer un nouveau attribut de ratio d'accès aux matériaux
material_access_ratio = Test['obs_accessible'].apply(lambda x: 1 if x=='Yes' else 0) / 12000

# Créer un nouveau attribut de ratio d'accès à l'eau potable
water_access_ratio = Test['obs_potable'].apply(lambda x: 1 if x=='Yes' else 0) / 12000

# Créer un nouveau attribut de ratio d'accès à l'énergie
energy_access_ratio = (Test['obs_heating_census'].apply(lambda x: 1 if x!='None' else 0) + \
                                Test['obs_cooking_census'].apply(lambda x: 1 if x!='None' else 0) + \
                                Test['obs_lighting_census'].apply(lambda x: 1 if x!='None' else 0)) / \
                               12000  

teacher_experiencee = Test['teacher_duration'].apply(lambda x: experience_bins(x))

"""social_rating = Test['teacher_social_total'].apply(lambda x: social_rating_bins(x))

age_appropriate_books = Test['obs_books_age'].apply(lambda x: 1 if x == 'Yes' else 0)

books_available = Test['obs_books'].map({'Yes': 1, 'No': 0, 'Unknown': -1})"""


""
# Get the index of the target attribute
target_index = Train.columns.get_loc('target')

# Insert the new attributes before the target attribute
Train.insert(loc=target_index, column='book_access_ratio', value=book_access_ratio)
Train.insert(loc=target_index+1, column='potable_water', value=potable_water)
Train.insert(loc=target_index+2, column='material_access_ratio', value=material_access_ratio)
Train.insert(loc=target_index+3, column='water_access_ratio', value=water_access_ratio)
Train.insert(loc=target_index+4, column='energy_access_ratio', value=energy_access_ratio)
Train.insert(loc=target_index+5, column='teacher_experience', value=teacher_experiencee)
"""Train.insert(loc=target_index+6, column='social_rating', value=social_rating)
Train.insert(loc=target_index+7, column='age_appropriate_books', value=age_appropriate_books)
Train.insert(loc=target_index+8, column='books_available', value=books_available)"""


# Insert the new attributes before the target attribute
Test.insert(loc=len(Test.columns), column='book_access_ratio', value=book_access_ratio)
Test.insert(loc=len(Test.columns), column='potable_water', value=potable_water)
Test.insert(loc=len(Test.columns), column='material_access_ratio', value=material_access_ratio)
Test.insert(loc=len(Test.columns), column='water_access_ratio', value=water_access_ratio)
Test.insert(loc=len(Test.columns), column='energy_access_ratio', value=energy_access_ratio) 
Test.insert(loc=len(Test.columns), column='teacher_experience', value=teacher_experiencee)
"""Test.insert(loc=len(Test.columns), column='social_rating', value=social_rating)
Test.insert(loc=len(Test.columns), column='age_appropriate_books', value=age_appropriate_books)
Test.insert(loc=len(Test.columns), column='books_available', value=books_available)"""





"Test.insert(loc=len(Test.columns), column='social_rating', value=social_rating)\nTest.insert(loc=len(Test.columns), column='age_appropriate_books', value=age_appropriate_books)\nTest.insert(loc=len(Test.columns), column='books_available', value=books_available)"

In [18]:
features = []; cat_features = []; not_features = []
for k in Train.columns[1:]:
    if Train[k].isnull().sum() < 6000:
        features.append(k)
        if Train[k].dtype == 'O':
            cat_features.append(k)
            print('There is '+ str(len(Train[k].value_counts()))+' Class in: ' +k)
    else:
        not_features.append(k)

print('----------------------------------')
print('We have '+str(len(features)) + ' features')
print('We have '+str(len(cat_features)) + ' categorical features')
print('We have '+str(len(not_features)) + ' features that have more than 6000 of missing values')

There is 254 Class in: child_date
There is 535 Class in: child_enrolment_date
There is 4 Class in: child_grant
There is 4 Class in: child_years_in_programme
There is 4 Class in: child_observe_attentive
There is 4 Class in: child_observe_concentrated
There is 4 Class in: child_observe_diligent
There is 4 Class in: child_observe_interested
There is 2 Class in: child_gender
There is 1018 Class in: child_dob
There is 3 Class in: child_stunted
There is 4 Class in: child_age_group
There is 153 Class in: id_mn_best
There is 10 Class in: prov_best
There is 50 Class in: id_dc_best
There is 50 Class in: dc_best
There is 153 Class in: mn_best
There is 5 Class in: pra_free_play
There is 4 Class in: pra_free_play_outdoor
There is 31 Class in: pra_groupings
There is 2 Class in: pra_groupings_1
There is 2 Class in: pra_groupings_2
There is 2 Class in: pra_groupings_3
There is 2 Class in: pra_groupings_4
There is 2 Class in: pra_groupings_5
There is 3 Class in: pra_engaged
There is 4 Class in: pra_age

In [19]:

Train = Train[features]
Test  = Test[features[:-1]]

In [20]:
Train = Train.fillna('')
Test  = Test.fillna('')

In [21]:
max(Train['target']), min(Train['target'])

(96.80999755859376, 6.369999885559082)

In [22]:
xtrain, xvalid, ytrain, yvalid = train_test_split(
    Train[Train.columns[:-1]],
    Train[Train.columns[-1]],
    test_size = 0.15,
    random_state = 42,
    shuffle = True
)

In [23]:
xtrain.shape, xvalid.shape, ytrain.shape, yvalid.shape

((7297, 288), (1288, 288), (7297,), (1288,))

In [24]:
xtest = Test[features[:-1]]
train_dataset = Pool(data = xtrain, label = ytrain, cat_features=cat_features)
val_dataset   = Pool(data = xvalid, label = yvalid, cat_features=cat_features)
model         = CatBoostRegressor(iterations = 30000, learning_rate=0.1, random_seed=123, verbose=300)
model.fit(train_dataset, eval_set=val_dataset, use_best_model=True, early_stopping_rounds=300)
preds_valid = model.predict(xvalid)
preds_test  = model.predict(xtest)
print(np.sqrt(mean_squared_error(yvalid, preds_valid)))

CatBoostError: Invalid type for cat_feature[non-default value idx=5,feature_idx=123]=0.0 : cat_features must be integer or string, real number values and NaN values should be converted to string.